In [78]:
client_id = ""
client_secret = ""
naver_id = ""
naver_pw = ""

In [224]:
%%time

import os
import sys
import urllib.request
import requests

import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm import notebook
import pickle
import re
import ast

from bs4 import BeautifulSoup
import urllib
import time

news_data = []
page_count = 49

# keywords = "에비드넷"
keywords = "웰빙 & 음식 & 건강 & 영양 & 장수"

encText = urllib.parse.quote(keywords)

for idx in notebook.tqdm(range(page_count)):
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&start=" + str(idx *10 +1)
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    if(rescode==200):
        #response_body = response.read()
        result = requests.get(response.geturl(),
                         headers={"X-Naver-Client-Id":client_id,"X-Naver-Client-Secret":client_secret})
        news_data.append(result.json())
    else:
        print("Error Code:" + rescode)
    time.sleep(1)    

    
naver_news_link = []

for page in news_data:
    page_news_link=[]
    for item in page['items']:
        link = item['link']
        #page_news_link.append(link)
        if "naver" in link:
            page_news_link.append(link)
    naver_news_link.append(page_news_link)        
            
link_count = 0        
for page in naver_news_link:
     for link in page:
            link_count += 1 
           #print(link)
        
#print(link_count)        
            


naver_news_title = []
naver_news_content = []

for n in notebook.tqdm(range(len(naver_news_link))):
    news_page_title = []
    news_page_content = []
    
    if n==0 or n%5 == 0:
        print("login")
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--window-size=1420,1080')
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(chrome_options=chrome_options)
        driver.get("https://nid.naver.com/nidlogin.login")
        driver.find_element_by_name('id').send_keys(naver_id)
        driver.find_element_by_name('pw').send_keys(naver_pw)
    
    for idx in notebook.tqdm(range(len(naver_news_link[n]))):
        time.sleep(2)

        try:
            
            driver.get(naver_news_link[n][idx])
        except:
            print("Timeout!")
            continue
        
        try:
            response = driver.page_source
        
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print("게시글이 삭제됨")
            continue
        
        soup = BeautifulSoup(response, "html.parser")
        
        title = None
        
        try:
            item = soup.find('div', class_="article_info")
            title = item.find('h3', class_="tts_head").get_text()
        except:
            title = "OUTLINK"
            
        news_page_title.append(title)
        
        doc = None
        text= ""
        
        data = soup.find_all("div",{"class" : "_article_body_contents"})
        if data:
            for item in data:
                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text)
                doc = ' '.join(text)
        else:
            doc = "OUTLINK"
        
        news_page_content.append(doc.replace('\n',' '))

    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)
                                  
with open("naver_news_title.pk","wb") as f:
    pickle.dump(naver_news_title, f)

with open("naver_news_content.pk", "wb") as f:
    pickle.dump(naver_news_content, f)        
        

login



login



login



login


Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!



Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!

login


Timeout!



Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!



Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!
Timeout!


CPU times: user 31.6 s, sys: 1.2 s, total: 32.8 s
Wall time: 13min 38s


In [225]:
from konlpy.tag import Mecab

mecab = Mecab()
import matplotlib.pyplot as plt
%matplotlib inline
# notebook.tqdm
from konlpy.tag import Mecab
import string
import warnings
from gensim import corpora
from gensim import models

warnings.filterwarnings("ignore",category=DeprecationWarning)

def read_documents(input_file_name):
    corpus = []
    
    with open(input_file_name,'rb') as f:
        temp_corpus = pickle.load(f)
        
    for page in temp_corpus:
        corpus += page
    
    return corpus

def text_cleaning(docs):
    cleaned_docs = []
    
    for doc in docs:
        temp_doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힇 ]","",doc)    
        cleaned_docs.append(temp_doc)
    return cleaned_docs

def define_stopwords(path):
    SW = set()
    for i in string.punctuation:
        SW.add(i)
        
    with open(path) as f:
        for word in f:
            SW.add(word)
    return SW

def text_tokenizing(corpus, tokenizer):
    #list  comprehension을 풀어서 쓴 코드
    #tokenized_doc = []
    #for word in mecab.morphs(doc):
    #    if word not in SW and len(word) > 1:
    #        tokenized_doc.append(word)
    #return tokenized_doc
    
    mecab = Mecab()
    token_corpus = []
    
    if tokenizer == "noun":
        for n in notebook.tqdm(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.nouns(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
    elif tokenized == "morph":
        for n in notebook.tqdm(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
    elif tokenized == "word":
        for n in notebook.tqdm(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
            
    return token_corpus

input_file_name = "naver_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("stopwords-ko.txt")
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun")

dictionary = corpora.Dictionary(tokenized_text)

corpus = [dictionary.doc2bow(text) for text in tokenized_text]

model = models.ldamodel.LdaModel(corpus, num_topics=link_count, id2word=dictionary)

#model.show_topic(0, 10)


In [226]:
NUM_TOPICS = link_count 
NUM_TOPIC_WORDS = 30 

def build_doc_term_mat(documents):
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
    return corpus, dictionary

def print_topic_words(model):
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic Id: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
        print("\n")
        

In [213]:
# document-term matrix
corpus, dictionary = build_doc_term_mat(tokenized_text)
# LDA 실행
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)

Topic Id: 0
	음식	0.01075730286538601
	건강	0.006867240183055401
	식품	0.005676181521266699
	비타민	0.004757621791213751
	플레이어	0.004612678661942482
	식초	0.004324090201407671
	내용	0.00409673573449254
	성분	0.00405912334099412
	푸드	0.0039682150818407536
	제품	0.003943599294871092
	본문	0.0038549345917999744
	요리	0.0035356797743588686
	효과	0.0032993420027196407
	때문	0.003265206702053547
	예방	0.0031141205690801144
	카레	0.0030354002956300974
	소비자	0.0029998209793120623
	대표	0.0029690752271562815
	시장	0.002938356949016452
	우리	0.002900966675952077
	사람	0.002886078553274274
	웰빙	0.0028422214090824127
	한국	0.0028320294804871082
	버섯	0.002698948374018073
	장수	0.002697383053600788
	재료	0.0026191482320427895
	국내	0.002566580194979906
	이상	0.0024601651821285486
	메뉴	0.0024058613926172256
	영양	0.0023302449844777584


Topic Id: 1
	건강	0.013553067110478878
	음식	0.010543245822191238
	식품	0.009203222580254078
	웰빙	0.004761666525155306
	우리	0.0046293893828988075
	본문	0.004267890937626362
	요리	0.004206744022667408
	내용	0.004107088316231966
	사람	0.00

In [227]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
55    -0.055999  0.093397       1        1  5.132740
179   -0.072629  0.031634       2        1  2.602734
71    -0.082164  0.018466       3        1  2.402562
98    -0.079147  0.036361       4        1  2.271989
240   -0.082135  0.015356       5        1  2.203596
...         ...       ...     ...      ...       ...
203   -0.016427 -0.041571     278        1  0.000785
202    0.086061 -0.060788     279        1  0.000785
128    0.025805 -0.083722     280        1  0.000785
201    0.073876 -0.027441     281        1  0.000785
140    0.080085 -0.015191     282        1  0.000785

[282 rows x 5 columns], topic_info=     Term         Freq        Total  Category  logprob  loglift
4      건강  1189.000000  1189.000000   Default  30.0000  30.0000
58     식품  1031.000000  1031.000000   Default  29.0000  29.0000
78     음식  1037.000000  1037.000000   Default  28.0000  28.0000
316    제품   496.000000   496.000000   Default  27.0000  27.0000
499    푸드   386.000000   386.000000   Default  26.0000  26.0000
..    ...          ...          ...       ...      ...      ...
0      가격     0.002210   115.157295  Topic282  -5.9642   0.8936
129    가능     0.002222   125.142578  Topic282  -5.9589   0.8158
15     내용     0.002736   482.735382  Topic282  -5.7506  -0.3259
116  플레이어     0.002531   448.962799  Topic282  -5.8286  -0.3314
38     본문     0.002270   456.454407  Topic282  -5.9376  -0.4570

[20178 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.008684   가격
0         2  0.026051   가격
0         3  0.017368   가격
0         4  0.026051   가격
0         5  0.008684   가격
...     ...       ...  ...
3097     20  0.356010   희로
3097     91  0.356010   희로
3250      3  0.272707  흰머리
927      15  0.158818   흰쌀
927      37  0.158818   흰쌀

[32007 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[56, 180, 72, 99, 241, 34, 214, 179, 234, 4, 90, 48, 253, 101, 108, 224, 260, 53, 177, 225, 176, 248, 164, 160, 112, 92, 16, 73, 215, 198, 23, 272, 37, 212, 211, 110, 268, 131, 104, 200, 35, 11, 43, 229, 143, 275, 183, 102, 65, 12, 125, 273, 128, 222, 85, 171, 189, 243, 170, 282, 126, 82, 2, 255, 130, 13, 7, 146, 6, 236, 245, 145, 206, 93, 210, 100, 244, 172, 127, 226, 187, 94, 51, 264, 271, 168, 60, 207, 113, 71, 201, 77, 247, 261, 156, 38, 45, 63, 230, 50, 150, 49, 1, 59, 54, 277, 254, 68, 166, 242, 22, 269, 18, 41, 95, 78, 17, 3, 178, 265, 32, 36, 98, 114, 165, 233, 31, 151, 267, 44, 119, 67, 217, 40, 194, 30, 116, 259, 250, 122, 25, 223, 117, 238, 26, 191, 163, 61, 115, 256, 28, 149, 80, 52, 97, 219, 118, 14, 167, 91, 5, 123, 276, 246, 262, 15, 66, 81, 84, 57, 257, 20, 197, 75, 109, 135, 9, 124, 29, 258, 227, 239, 188, 70, 152, 64, 249, 58, 62, 181, 42, 55, 21, 280, 279, 278, 8, 10, 274, 270, 19, 24, 251, 27, 33, 266, 263, 39, 46, 47, 252, 69, 232, 240, 144, 154, 153, 148, 147, 195, 196, 142, 133, 281, 140, 139, 138, 137, 136, 155, 193, 157, 158, 159, 192, 161, 162, 190, 169, 186, 185, 173, 174, 175, 184, 182, 134, 132, 74, 88, 220, 221, 96, 228, 231, 89, 87, 199, 86, 83, 235, 79, 237, 76, 103, 218, 105, 106, 107, 216, 111, 213, 209, 120, 121, 208, 205, 204, 203, 129, 202, 141])

In [228]:
print(model.num_topics)

282
